# Lucena - Data Collection

## Finding Realease with most bugs

In [20]:
import json
from pprint import pprint
from os import listdir
from os.path import isfile, join
import operator

mypath = "issue_LUCENE"

onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) and f.endswith(".json") ]
version_count = {}

for f in onlyfiles:
    with open(join(mypath,f)) as data_file:   
        data = json.load(data_file)
        if (data["fields"]["issuetype"]["name"] == "Bug" and 
            data["fields"]["status"]["name"] == "Closed" and 
            data["fields"]["resolution"]["name"] == "Fixed"):
    
            versions = data["fields"]["versions"]
            for v in versions:
                if version_count.has_key(v["name"]):
                    version_count[v["name"]].append(data["key"])
                else:
                    version_count[v["name"]] = [data["key"]]
        


From the dictionary with the bugs associated to each release we extract the buggiest version and its bugs.

In [35]:
max = 0
buggy_version = ""

for k,v in version_count.iteritems():
    if len(v) > max:
        max = len(v)
        buggy_version = k
        
bugs = {}

for b in version_count[buggy_version]:
    bugs[b] = []

print "Version with most closed bugs:", buggy_version , "with", len (version_count[buggy_version]), "bugs."

{u'LUCENE-4282': [], u'LUCENE-3476': [], u'LUCENE-4125': [], u'LUCENE-3575': [], u'LUCENE-2736': [], u'LUCENE-2737': [], u'LUCENE-2534': [], u'LUCENE-3672': [], u'LUCENE-2645': [], u'LUCENE-4114': [], u'LUCENE-2467': [], u'LUCENE-3324': [], u'LUCENE-3327': [], u'LUCENE-3329': [], u'LUCENE-3777': [], u'LUCENE-4119': [], u'LUCENE-4218': [], u'LUCENE-3768': [], u'LUCENE-3880': [], u'LUCENE-3561': [], u'LUCENE-2703': [], u'LUCENE-2705': [], u'LUCENE-3251': [], u'LUCENE-2936': [], u'LUCENE-3430': [], u'LUCENE-4209': [], u'LUCENE-4364': [], u'LUCENE-3600': [], u'LUCENE-4071': [], u'LUCENE-4070': [], u'LUCENE-3095': [], u'LUCENE-4075': [], u'LUCENE-4074': [], u'LUCENE-3090': [], u'LUCENE-3608': [], u'LUCENE-4162': [], u'LUCENE-3398': [], u'LUCENE-3800': [], u'LUCENE-2970': [], u'LUCENE-3452': [], u'LUCENE-2344': [], u'LUCENE-3743': [], u'LUCENE-3692': [], u'LUCENE-2717': [], u'LUCENE-2668': [], u'LUCENE-3062': [], u'LUCENE-3895': [], u'LUCENE-3590': [], u'LUCENE-2796': [], u'LUCENE-3068': [],

For some reason shortlog was not working. #git shortlog -s -n

In [26]:
import sh
import re

#sh.git.clone("https://github.com/apache/lucene-solr.git")
git = sh.git.bake(_cwd='lucene-solr')

git.checkout("tags/lucene_solr_4_0_0_ALPHA")


files = filter(None,git("ls-files").split("\n"))

print "Reading repository..."

#git shortlog -s -n
#print sh.uniq(sh.sort(git.log("--format=format:%an", "README.txt")), "-c")

table = {}
count = 0 #TODO remove

for f in files:
    contributors_data = filter(None, sh.uniq(sh.sort(git.log("--format=format:%an", f)), "-c").split("\n"))
    contributors = []
    total = 0
    max_ownership = 0
    minor = 0
    major = 0
    
    for a in contributors_data:
        num = int(re.search("[0-9]+", a).group(0))
        name = re.search("([A-z]+\s*)+", a).group(0)
        total += num
        max_ownership = num if num > max_ownership else max_ownership
        contributors.append((name,num))
    
    for a in contributors:

        if a[1] * 1.0 / total >= 0.05:
            major += 1
        else:
            minor += 1
   
    count += 1 #TODO delete
    if count >= 100: #TODO delete
        break #TODO delete
        
    table[f]= {"minor": minor, "major": major, "total": minor + major, "ownership": max_ownership / total * 100, "bug_count":0}
    #print table[f]

print table

print "finish"


Reading repository...
{u'dev-tools/maven/lucene/facet/pom.xml.template': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'dev-tools/eclipse/dot.project': {'bug_count': 0, 'major': 1, 'total': 1, 'ownership': 100, 'minor': 0}, u'dev-tools/idea/lucene/analysis/stempel/stempel.iml': {'bug_count': 0, 'major': 1, 'total': 1, 'ownership': 100, 'minor': 0}, u'dev-tools/maven/lucene/join/pom.xml.template': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'dev-tools/maven/lucene/analysis/stempel/pom.xml.template': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'dev-tools/idea/.idea/libraries/ICU_library.xml': {'bug_count': 0, 'major': 2, 'total': 2, 'ownership': 0, 'minor': 0}, u'dev-tools/idea/.idea/libraries/Solr_DIH_library.xml': {'bug_count': 0, 'major': 1, 'total': 1, 'ownership': 100, 'minor': 0}, u'dev-tools/idea/.idea/projectCodeStyle.xml': {'bug_count': 0, 'major': 1, 'total': 1, 'ownership': 100, 'minor': 0}, u'dev-t

## Finding commits that fix bugs of the chosen release

In [65]:
#git rev-list --topo-order HEAD..towards | tail -1
#git rev-parse trunk
last_commit_hash = git("rev-parse", "trunk")
print last_commit_hash

commit_list = filter(None, git("rev-list", "--topo-order", "HEAD.." + str(last_commit_hash).strip()).split("\n"))
print len(commit_list)

# git log --format=%B -n 1 7bfa4fd6505e86c7481526bca7b157055a7b4ead
count = 0

print 

for commit in commit_list:
    message = str(git.log("--format=%B", "-n 1", commit))
    match = re.search("LUCENE-[0-9]+", message)

    if match:
        key = match.group(0).strip()
        
        if key in bugs: 
            count += 1
            print key
            files_changed = filter(None, git("diff-tree", "--no-commit-id", "--name-only", "-r", commit).split("\n"))
            bugs[key] = files_changed
   
print "Identified fixes for", str(count), "out of", str(len(bugs)) + "."

print bugs

#print key, "-->", message


7bfa4fd6505e86c7481526bca7b157055a7b4ead

10113

LUCENE-3849
LUCENE-3849
LUCENE-4801
LUCENE-4333
LUCENE-4364
LUCENE-4364
LUCENE-4364
LUCENE-4364
LUCENE-4218
LUCENE-4342
LUCENE-4342
LUCENE-4333
LUCENE-4333
LUCENE-4302
LUCENE-4282
LUCENE-4266
LUCENE-4234
LUCENE-4221
LUCENE-4209
LUCENE-4209
LUCENE-4209
LUCENE-4185
LUCENE-4080
LUCENE-4178
LUCENE-4158
LUCENE-4162
LUCENE-4147
LUCENE-4158
LUCENE-4149
LUCENE-4137
LUCENE-4135
LUCENE-4142
LUCENE-4139
LUCENE-4130
LUCENE-4129
LUCENE-4127
LUCENE-4116
LUCENE-4127
LUCENE-4119
LUCENE-4114
LUCENE-4103
LUCENE-4111
LUCENE-4090
Identified fixes for 43 out of 179.
{u'LUCENE-4282': [u'lucene/CHANGES.txt', u'lucene/core/src/java/org/apache/lucene/search/FuzzyTermsEnum.java', u'lucene/core/src/test/org/apache/lucene/search/TestFuzzyQuery.java', u'lucene/sandbox/src/test/org/apache/lucene/sandbox/queries/TestSlowFuzzyQuery.java'], u'LUCENE-3476': [], u'LUCENE-4125': [], u'LUCENE-3575': [], u'LUCENE-2736': [], u'LUCENE-2737': [], u'LUCENE-2534': [], u'LUCENE-36